In [19]:
import pandas as pd
import numpy as np
import xgboost as xgb
import math
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [20]:
input_dir = '../data/'
train_data = pd.read_csv(input_dir + 'train.csv')
test_data = pd.read_csv(input_dir + 'test.csv')

In [21]:
train.head(10)

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v123,v124,v125,v126,v127,v128,v129,v130,v131,cnt
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144,2
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN,81
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472,3
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483,2
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN,103
5,9,0,NaN,NaN,C,NaN,8.856791,NaN,NaN,0.359993,...,NaN,0.049861,X,NaN,NaN,1.536222,0,NaN,NaN,83
6,12,0,0.899806,7.312995,C,3.494148,9.946200,1.926070,1.770427,0.066251,...,1.992594,0.083758,BJ,3.276100,1.623298,2.266575,0,2.263736,0.970873,2
7,21,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,BY,NaN,NaN,NaN,0,NaN,NaN,101
8,22,0,2.078651,8.462619,NaN,3.739030,5.265636,1.573033,2.303371,0.015869,...,1.875560,0.018659,S,1.159637,5.582865,1.105283,0,1.170731,3.333334,5
9,23,1,1.144802,5.880606,C,3.244469,9.538384,2.500001,1.559405,0.412610,...,4.896617,0.008944,E,1.344550,1.601176,1.928009,0,3.174603,1.000000,2


In [22]:
cat_cols = ['v22', 'v91', 'v3', 'v24', 'v30', 'v31', 'v47', 'v52', 'v56', 'v66', 'v71', 
            'v74', 'v75', 'v79', 'v107', 'v110', 'v112', 'v113', 'v125']
all_cols = train_data.columns.values[2:]

In [24]:
all_data = train_data.append(test_data)
all_data['count_nan'] = all_data.isnull().sum(axis=1)

In [25]:
for col in cat_cols:
    all_data[col] = pd.factorize(all_data[col])[0]
all_data.fillna(-1, inplace=True)

In [28]:
all_data['target'] = all_data['target'].astype(int)
all_data[all_cols] = all_data[all_cols].astype(float)

In [29]:
train = all_data[all_data['target'] > -1].copy()
test = all_data[all_data['target'] == -1].copy()

In [30]:
dtrain = xgb.DMatrix(train.drop(['ID', 'target'], axis=1), train['target'].values)
dtest = xgb.DMatrix(test.drop(['ID', 'target'], axis=1), label=test['target'].values)

In [34]:
params = {}
params['objective'] = 'binary:logistic'
params['eta'] = 0.01
params['min_child_weight'] = 1
params['subsample'] = 0.9
params['colsample_bytree'] = 0.8
params['silent'] = 1
params['max_depth'] = 10
params['eval_metric'] = 'logloss'
params['nthread'] = -1
num_rounds = 10

In [35]:
model = xgb.train(params, dtrain, num_rounds)

In [41]:
y_pred = model.predict(dtest)

In [42]:
pd.DataFrame({'ID': test['ID'].values, 'PredictedProb': y_pred}).to_csv('submit.csv', index=False)